In [1]:
!wget https://github.com/Bao02Harry/Recognize_Rock_Paper_Scissors_Application/blob/main/SmallRPSDataset-master.zip?raw=true

--2022-05-26 15:34:10--  https://github.com/Bao02Harry/Recognize_Rock_Paper_Scissors_Application/blob/main/SmallRPSDataset-master.zip?raw=true
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Bao02Harry/Recognize_Rock_Paper_Scissors_Application/raw/main/SmallRPSDataset-master.zip [following]
--2022-05-26 15:34:10--  https://github.com/Bao02Harry/Recognize_Rock_Paper_Scissors_Application/raw/main/SmallRPSDataset-master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Bao02Harry/Recognize_Rock_Paper_Scissors_Application/main/SmallRPSDataset-master.zip [following]
--2022-05-26 15:34:11--  https://raw.githubusercontent.com/Bao02Harry/Recognize_Rock_Paper_Scissors_Application/main/SmallRPSDataset-master.zip
Resolving raw.githubusercontent.com (r

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import matplotlib.image as mplimg

In [3]:
import zipfile
local_zip = './SmallRPSDataset-master.zip?raw=true'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./SmallRPSDataset-master')
zip_ref.close()

In [4]:
# Directory with our training rock picture
train_rock_dir = os.path.join("./SmallRPSDataset-master/SmallRPSDataset-master/rock")
# Directory with our training paper picture
train_paper_dir = os.path.join("./SmallRPSDataset-master/SmallRPSDataset-master/paper")
# Directory with our training scissors picture
train_scissors_dir = os.path.join("./SmallRPSDataset-master/SmallRPSDataset-master/scissors")

In [5]:
train_rock_names = os.listdir(train_rock_dir)
train_paper_names = os.listdir(train_paper_dir)
train_scissors_names = os.listdir(train_scissors_dir)

In [6]:
print(len(train_rock_names))
print(len(train_paper_names))
print(len(train_scissors_names))

299
299
299


In [16]:
# Build model
model= tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (300, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # This is the second convolution
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # This is the third convolution 
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # This is the forth convolution
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    # This is the fifth convolution convolution
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.05),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation = 'relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horse) and 1 for the orther('humans')
    tf.keras.layers.Dense(3, activation = 'softmax')
])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 298, 198, 32)      896       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 149, 99, 32)      0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 147, 97, 32)       9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 73, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 71, 46, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 35, 23, 64)      

In [18]:
# All images will be rescaled by 1/255
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    './SmallRPSDataset-master/SmallRPSDataset-master/', # This is the source directory for training images
    target_size= (300, 200), # All images will be resized to 300x300
    batch_size = 32
)


Found 897 images belonging to 3 classes.


In [19]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('loss') <= 0.01):
      print("\nLoss is lower than 0.03 so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()


In [20]:
model.compile(loss='mse',
              optimizer= 'adam',
              metrics=['accuracy'])

In [21]:
seed = 20
np.random.seed(seed)
history = model.fit(train_generator, epochs=20, callbacks =[callbacks])

Epoch 1/20
29/29 [==============================] - 4s 107ms/step - loss: 0.1751 - accuracy: 0.5708
Epoch 2/20
29/29 [==============================] - 3s 103ms/step - loss: 0.0758 - accuracy: 0.8573
Epoch 3/20
29/29 [==============================] - 3s 104ms/step - loss: 0.0300 - accuracy: 0.9476
Epoch 4/20
29/29 [==============================] - 3s 102ms/step - loss: 0.0282 - accuracy: 0.9509
Epoch 5/20
29/29 [==============================] - 3s 102ms/step - loss: 0.0161 - accuracy: 0.9710
Epoch 6/20
29/29 [==============================] - 3s 106ms/step - loss: 0.0161 - accuracy: 0.9721
Epoch 7/20
29/29 [==============================] - 3s 105ms/step - loss: 0.0122 - accuracy: 0.9755
Epoch 8/20
29/29 [==============================] - 3s 110ms/step - loss: 0.0122 - accuracy: 0.9777
Epoch 9/20
29/29 [==============================] - 3s 105ms/step - loss: 0.0125 - accuracy: 0.9766
Epoch 10/20
29/29 [==============================] - ETA: 0s - loss: 0.0050 - accuracy: 0.9911
Loss 

In [22]:
tf.keras.models.save_model(model, filepath = '/content/model.h5')

In [28]:

from google.colab import files
from keras.preprocessing import image
uploaded = files.upload()
# uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 200))
  x = image.img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes)
  max_index_row = np.argmax(np.array(classes), axis=1)
  print(max_index_row)